In [3]:
# 여러게시물 #상세내용 #수집 웹 크롤러 
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

#1. 크롤링 기본 셋팅하기
print("=" *80)
print("연습문제 6-2 : 청와대 국민청원 게시판 크롤링하기")
print("=" *80)
query_txt = '청와대국민청원'


cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 7)



f_dir = input("2.결과 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): ")
if f_dir=='' :
    f_dir='/Users/kibeomkim/Desktop/result_data/'

    
# 저장될 파일위치와 이름 지정하기
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name= s+'-'+query_txt+'.txt'
fc_name= s+'-'+query_txt+'.csv'
fx_name= s+'-'+query_txt+'.xls'


# 크롬 드라이버를 사용해서 웹 브라우저 실행하기 

s_time = time.time( )

path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
driver = webdriver.Chrome(path)

driver.get('https://www1.president.go.kr/petitions/?c=0&only=1&page=1&order=1')




time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

pet_no2 = []

no=1

# 페이지를 변경하면서 사용자가 요청한 건수만큼 내용을 추출하여 파일에 저장하기


pet_title2=[]
pet_people=[]
pet_cat=[]
pet_dog=[] #중간 리스트
pet_start=[]
pet_end=[]
pet_contents2=[]     # 게시글 내용 컬럼

import sys 
original_stdout = sys.stdout
f = open(ff_name, 'a', encoding='UTF-8')
sys.stdout = f

for x in range(1,page_cnt+1) :    
    #청원번호 리스트 만들기
    html0 = driver.page_source 
    soup0 = BeautifulSoup(html0, 'html.parser')
    
    pet_no1 = soup0.find('ul', 'petition_list')
    aux = pet_no1.find_all('a', 'cb relpy_w')



    for i in range(1,8):  
        print("%s 번째 국민청원 게시글 상세 정보입니다 =======================" %i)
        print('\n')

    
        if no > cnt :
            break
        
        pig = aux[i-1]
        pig2 = pig['href']
        n_url = pig2.split('/')[2]
        pet_no2.append(n_url)
        
        
        
        f.write('1. 청원번호:'+pet_no2[i-1]+'\n')
        
        driver.find_element_by_xpath('/html/body/div[1]/div/div[3]').click()
        time.sleep(1.5)
        

        #각 게시글의 제목 누르기    
        driver.find_element_by_xpath('/html/body/div[3]/div[2]/section[2]/div[2]/div/div/div[2]/div[2]/div[4]/div/div[2]/div[2]/ul/li[%s]/div/div[3]' %i).click()
        time.sleep(2)
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')   #파싱완료

        #청원제목
        content_title = soup.find('div', 'petitionsView_left').find('h3', 'petitionsView_title')
        title_1 = content_title.get_text().strip()
        f.write('2. 청원제목:'+title_1+'\n')
        pet_title2.append(title_1)
        
        #참여인원
        participant_n = soup.find('h2', 'petitionsView_count').find('span', 'counter')
        parti_1 = participant_n.get_text().strip()
        f.write('3. 참여인원:'+parti_1+'\n')
        pet_people.append(parti_1)

        #카테고리-1
        view_list = soup.find('ul', 'petitionsView_info_list').find_all('li')
        for a in view_list : 
            pet_dog.append(a.get_text())
            
        
        #카테고리-2
        cate_2 = pet_dog[0].replace("\n", "")
        cate_3 = cate_2.replace("카테고리", "").strip()
        f.write('4. 카테고리:'+cate_3+'\n')
        pet_cat.append(cate_3)
        
        
        
        #청원시작일 
        p_start = pet_dog[1].replace('\n', "")
        p_start_1 = p_start.replace("청원시작", "").strip()
        f.write('5. 청원시작일:'+p_start_1+'\n')
        pet_start.append(p_start_1)
        
        
        #청원종료일 
        p_end = pet_dog[2].replace('\n', "")
        p_end_1 = p_end.replace("청원마감", "").strip()
        f.write('6. 청원종료일:'+p_end_1+'\n')
        pet_end.append(p_end_1)
        
        
        #청원내용 
        p_contents = soup.find('div', 'petitionsView_write').find('div', 'View_write')
        p_contents1 = p_contents.get_text().replace('\n', "").strip()
        f.write('7. 청원내용:'+'\n')
        f.write(p_contents1 +'\n')
        f.write('\n')
        pet_contents2.append(p_contents1)
        

        

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(2)
                
        no += 1
        
        if no > cnt :
            break
        

    x += 1
    
    if x > page_cnt :
        break
    else :
        try :
            driver.find_element_by_link_text('%s' %x).click() # 다음 페이지번호 클릭              
        except :
            try : 
                driver.find_element_by_link_text('Next').click()
                time.sleep(2)
            except : 
                break
  
              
    time.sleep(2)
    
f.close()
sys.stdout = original_stdout


 



# Step 8. 출력 결과를 표(데이터 프레임) 형태로 만들어 csv,xls 형식으로 저장하기
korea = pd.DataFrame()
korea['청원번호']=pet_no2
korea['청원제목']=pet_title2
korea['참여자']=pet_people
korea['카테고리']=pet_cat
korea['청원시작일']=pet_start
korea['청원종료일']=pet_end
korea['청원내용']=pet_contents2



# csv 형태로 저장하기
korea.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
korea.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time



1.크롤링 할 건수는 몇건입니까?: 300
2.결과 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): 


In [4]:
# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )